<a href="https://colab.research.google.com/github/drfperez/DeepPurpose/blob/main/Gromacs4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mdtraj Bio gromacs
import requests
import mdtraj as md
import subprocess
import os

def fetch_combined_pdb_content(pdb_code):
    try:
        # Fetch the PDB file from RCSB PDB website
        pdb_url = f"https://files.rcsb.org/download/{pdb_code.lower()}.pdb"
        response = requests.get(pdb_url)

        # Check if the request was successful
        if response.status_code == 200:
            return response.text
        else:
            return None  # Return None if PDB code not found
    except Exception as e:
        return None  # Return None if there is an error

def convert_pdb_to_gromacs(pdb_content):
    try:
        # Write PDB content to a temporary file
        with open("temp.pdb", "w") as f:
            f.write(pdb_content)

        # Read PDB file using mdtraj
        traj = md.load("temp.pdb")

        # Save trajectory in GROMACS format
        traj.save("output.gro")

        # Remove temporary PDB file
        os.remove("temp.pdb")

        return True
    except Exception as e:
        print(f"Error: {e}")
        return False

def generate_topology():
    try:
        # Generate topology file using pdb2gmx
        subprocess.run(['gmx', 'pdb2gmx', '-f', 'output.gro', '-o', 'output_processed.gro', '-water', 'spce'], check=True)
        return True
    except Exception as e:
        print(f"Error: {e}")
        return False

def check_gromacs_installation():
    try:
        # Check GROMACS installation
        result = subprocess.run(['gmx', '--version'], capture_output=True, text=True)
        if result.returncode == 0 and "GROMACS version" in result.stdout:
            return True
        else:
            return False
    except Exception as e:
        print(f"Error checking GROMACS installation: {e}")
        return False

def install_gromacs():
    try:
        # Install GROMACS using apt-get
        subprocess.run(['apt-get', 'install', '-y', 'gromacs'], check=True)
        return True
    except Exception as e:
        print(f"Error installing GROMACS: {e}")
        return False

# Prompt the user to enter a PDB code
pdb_code = input("Enter PDB code for protein with ligand: ")

# Check GROMACS installation
if not check_gromacs_installation():
    print("GROMACS is not installed. Installing...")
    if install_gromacs():
        print("GROMACS installed successfully.")
    else:
        print("Error installing GROMACS. Aborting.")
        exit()

# Example usage:
pdb_content = fetch_combined_pdb_content(pdb_code)
if pdb_content is not None:
    print("PDB content downloaded successfully.")
    # Convert PDB to GROMACS format
    if convert_pdb_to_gromacs(pdb_content):
        print("PDB converted to GROMACS format successfully.")
        # Generate topology file
        if generate_topology():
            print("Topology file generated successfully.")
        else:
            print("Error generating topology file.")
    else:
        print("Error converting PDB to GROMACS format.")
else:
    print("Error: PDB code not found or failed to download.")

# Step 1: Energy Minimization
print("Step 1: Energy Minimization")
try:
    subprocess.run(['gmx', 'grompp', '-f', 'minim.mdp', '-c', 'output_processed.gro', '-p', 'topol.top', '-o', 'em.tpr'], check=True)
    subprocess.run(['gmx', 'mdrun', '-deffnm', 'em'], check=True)
    print("Energy minimization completed successfully.")
except Exception as e:
    print(f"Error performing energy minimization: {e}")

# Step 2: Equilibration
print("Step 2: Equilibration")
try:
    subprocess.run(['gmx', 'grompp', '-f', 'nvt.mdp', '-c', 'em.gro', '-r', 'em.gro', '-p', 'topol.top', '-o', 'nvt.tpr'], check=True)
    subprocess.run(['gmx', 'mdrun', '-deffnm', 'nvt'], check=True)
    subprocess.run(['gmx', 'grompp', '-f', 'npt.mdp', '-c', 'nvt.gro', '-r', 'nvt.gro', '-p', 'topol.top', '-o', 'npt.tpr'], check=True)
    subprocess.run(['gmx', 'mdrun', '-deffnm', 'npt'], check=True)
    print("Equilibration completed successfully.")
except Exception as e:
    print(f"Error performing equilibration: {e}")

# Step 3: Production Molecular Dynamics (MD)
print("Step 3: Production Molecular Dynamics (MD)")
try:
    subprocess.run(['gmx', 'grompp', '-f', 'md.mdp', '-c', 'npt.gro', '-t', 'npt.cpt', '-p', 'topol.top', '-o', 'md.tpr'], check=True)
    subprocess.run(['gmx', 'mdrun', '-deffnm', 'md'], check=True)
    print("Production MD completed successfully.")
except Exception as e:
    print(f"Error performing production MD: {e}")

# Step 4: Trajectory Analysis
print("Step 4: Trajectory Analysis")
try:
    # Example: RMSD calculation
    subprocess.run(['gmx', 'rms', '-s', 'md.tpr', '-f', 'md.xtc', '-o', 'rmsd.xvg'], check=True)
    print("RMSD calculation completed successfully.")
except Exception as e:
    print(f"Error performing RMSD calculation: {e}")

# Step 5: Free Energy Calculations
print("Step 5: Free Energy Calculations")
try:
    # Example: MM-PBSA calculation
    subprocess.run(['gmx', 'g_mmpbsa', '-s', 'md.tpr', '-f', 'md.xtc', '-n', 'index.ndx', '-pdie', '2'], check=True)
    print("MM-PBSA calculation completed successfully.")
except Exception as e:
    print(f"Error performing MM-PBSA calculation: {e}")

# Step 6: Visualize Results
print("Step 6: Visualize Results")
try:
    # Example: Visualize trajectory using VMD
    subprocess.run(['vmd', '-gro', 'md.gro', '-xtc', 'md.xtc'], check=True)
    print("Trajectory visualization completed successfully.")
except Exception as e:
    print(f"Error visualizing trajectory: {e}")

# Step 7: Binding Energy Calculation
print("Step 7: Binding Energy Calculation")
try:
    # Perform MM-PBSA calculation
    subprocess.run(['gmx', 'g_mmpbsa', '-s', 'md.tpr', '-f', 'md.xtc', '-n', 'index.ndx', '-pdie', '2'], check=True)
    print("Binding energy calculation (MM-PBSA) completed successfully.")
except Exception as e:
    print(f"Error performing binding energy calculation: {e}")

  Using cached mdtraj-1.9.9.tar.gz (2.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.8 MB/s eta 0:00:00
  Created wheel for mdtraj: filename=mdtraj-1.9.9-cp310-cp310-linux_x86_64.whl size=7547271 sha256=7011a926b510defda39e2472121970185e084f7bb60c4610650a42f7ebbc5561
  Stored in directory: /root/.cache/pip/wheels/4b/4e/d5/22b44e04aca0780281ffb43717ebb3404bc1a77cf8f345fa73
  Created wheel for gromacs: filename=gromacs-0.0.0-py3-none-any.whl size=1415 sha256=c8b35b3b1723303164719e241f1d3f93d356c85b64605ea853e2aa41384f3349
  Stored in directory: /root/.cache/pip/wheels/4b/18/21/7d9699d80505e3b2ae8eb2d49f9225f2e2e5c2e0ec2b0a3a46
Successfully built mdtraj gromacs
Error checking GROMACS installation: [Errno 2] No such f